In [1]:
import pandas as pd
from django.core.exceptions import ObjectDoesNotExist

In [2]:
# file = "legacy_data/WStLA_MD_Allgemeine_Registratur_Akten_gesamt_20180903.xlsx"
file = "legacy_data/20180901_VGA.xlsx"
res_type_scheme, _ = SkosConceptScheme.objects.get_or_create(dc_title='res_type')
archiv, _ = Institution.objects.get_or_create(
    written_name='Wiener Stadt- und Landesarchiv',
    abbreviation="WStLA",
    institution_type="Archiv"
)

In [11]:
df = pd.read_excel(file, dtype={'Datum original': str, 'Karton': str}).fillna('False')

In [10]:
df

,Teil von Bestand,(Archiv)Signatur,Karton,Aktenzahl,Betreff (Titel),Typ,Datum original,Datum normalisiert DD/MM/YYYY),Datum normalisiert nicht nach(DD/MM/YYYY),Personen,Institutionen,Orte,Schlagwörter,Verweis,Zusammenfassung,Kommentar
0,"VGA, Sozialdemokratische Parteistellen",1,"Kt. 4, M 33",keine,"Schreiben an den Bürgermeister in Donawitz, Jo...",Schreiben (Durchschlag),20. Nov. 1919,1919-11-20 00:00:00,False,Josef Heissl,Zentralstelle für sozialdemokratische Gemeinde...,Donawitz,Gehaltsschema; Dienstordnung; Angestellte (öff...,False,"Schreiben an den Bürgermeister in Donawitz, Jo...",False
1,"VGA, Sozialdemokratische Parteistellen",2,"Kt. 4, M 33",keine,"Schreiben an Dr. Julius Fischer, St. Pölten",Schreiben (Durchschlag),18. Okt. 1919,1919-10-18 00:00:00,False,Dr. Julius Fischer,Zentralstelle für sozialdemokratische Gemeinde...,St. Pölten,Dienstordnung; Pensionsnormale,False,"Schreiben an Dr. Julius Fischer, St. Pölten: E...",False
2,"VGA, Sozialdemokratische Parteistellen",3,"Kt. 4, M 34",keine,Schreiben der Sicherheitswache Donawitz an die...,Schreiben (Durchschlag),9. April 1920,1920-04-09 00:00:00,False,Josef Heissl; Hans Pöllitsch,Zentralstelle für sozialdemokratische Gemeinde...,Donawitz,Dienstordnung; Dienstpragmatik,False,Schreiben der Sicherheitswache Donawitz an die...,False
3,"VGA, Sozialdemokratische Parteistellen",4,"Kt. 23, M 123, Umschlag Interpellationen 1911/19",676,Aufnahme im Dienst der Gemeinde Wien als Sanit...,"Protokoll, maschinenschriftlich, mit Unterschr...",5. Juli 1919,1919-07-05 00:00:00,False,Adolf Jandourek; Hr. Vokner (Stationsleiter XI...,Gesundheitsamt der Stadt Wien,Wien,Koalitonsfreiheit; Streik; Heimkehrer; Sanität...,False,"Protokoll mit Saniätsdiener Adolf Jandourek, i...",False
4,"VGA, Sozialdemokratische Parteistellen",5,"Kt. 77, M 459",False,"Sitzung des Stadtratksklubs, am 5. November 19...","Protokoll, maschinenschriftlich, fortlaufend m...",5. November 1919,1919-11-05 00:00:00,False,Paul Speiser; Julius Kopriva; Ferdinand Skaret...,Klub der Sozialdemokraten im Wiener Gemeindera...,Wien,Lohnforderung; Gebührenerhöhung,Zu Nr. 7 & 10 in dieser Tabelle,Paul Speiser berichtet über die drängenden For...,False
5,"VGA, Sozialdemokratische Parteistellen",6,"Kt. 77, M 459",False,"Vollversammlung am 7. November 1919, 177f.","Protokoll, maschinenschriftlich, fortlaufend m...",7. November 1919,1919-11-07 00:00:00,False,Ferdinand Skaret; Hugo Breitner; Gabriele Prof...,Städtische Straßenbahnen; Klub der Sozialdemok...,Wien,Lohnforderung; Gebührenerhöhung,zu Nr. 6 in dieser Tabelle,Prorokoll der Vollversammlung mit der Tagesord...,False
6,"VGA, Sozialdemokratische Parteistellen",7,"Kt. 77, M 459",False,"Sitzung des Klubvorstandes, der Gewerkschaftsk...","Protokoll, maschinenschriftlich, fortlaufend m...",13. Oktober 1919,1919-10-13 00:00:00,False,Ferdinand Skaret; Hugo Breitner; Hueber; Josef...,Städtische Gaswerke; Städtische Straßenbahnen;...,WIen,Lohnforderung; Gebührenerhöhung; Personalkoste...,zu Nr. 9 in dieser Tabelle,Protokoll einer Sitzung des Klubvorstandes mit...,False
7,"VGA, Sozialdemokratische Parteistellen",8,"Kt. 77, M 459",False,Sitzung des Komitees zur Beratung der Lohnford...,"Protokoll, maschinenschriftlich, fortlaufend m...",16.Oktober 1919,1919-10-16 00:00:00,False,Ferdinand Skaret; Hugo Breitner; Ratzenberger;...,Städtische Gaswerke; Komitee zur Beratung der ...,Wien,Lohnforderung; Lohnverhandlung,zu Nr. 8 in dieser Tabelle,Protokoll einer Sitzung des Komitees zur Berat...,False
8,"VGA, Sozialdemokratische Parteistellen",9,"Kt. 77, M 459",False,Richtlinien für Kollektivverträge zwischen der...,False,5.November 1919,1919-11-05 00:00:00,False,False,Sozialdemokratischer Stadtsratsklub; Klub der ...,Wien,Kollektivvertrag,zu Nr. 6 in dieser Tabelle,Richtlinien für Kollektivverträge zwischen der...,False
9,"VGA, Sozialdemokratische Parteistellen",10,"Kt. 77, M 459",False,Protokoll des Stadtrats-Klub-Sitzung vom 15. N...,"Protokoll, handschriftlich",15.November 1919,1919-11-15

In [ ]:
for i, row in df.iterrows():
    item = None
    if row['Aktenzahl'] != 'False':
        aktenzahl = row['Aktenzahl']
    else:
        aktenzahl = 'keine'
    signatur = ", ".join(
        [
            "Bestand: {}".format(str(row.get('Teil von Bestand', default = "-"))), 
            "Karton: {}".format(str(row.get('Karton', default='-'))), 
            "Aktenzahl: {}".format(aktenzahl),
            "[internal-id]: {}".format(i)
        ]
    )
    try:
        item, _ = ArchResource.objects.get_or_create(
            signature=signatur
        )
    except Exception as e:
        print(e)
    if item:
        if row['Betreff (Titel)'] != 'False':
            item.title = row['Betreff (Titel)']
        if row['Zusammenfassung'] != 'False':
            item.abstract = row['Zusammenfassung']
        if row['Kommentar'] != 'False':
            item.notes = row['Kommentar']
        if row['Datum original '] != 'False':
            item.written_date = row['Datum original '].replace('00:00:00', '')
        if row['Datum normalisiert DD/MM/YYYY)'] != 'False':
            try:
                item.not_before = pd.to_datetime(row['Datum normalisiert DD/MM/YYYY)'])
            except Exception as e:
                print(e)
        if row['Datum normalisiert nicht nach(DD/MM/YYYY)'] != 'False':
            try:
                item.not_after = pd.to_datetime(row['Datum normalisiert nicht nach (DD/MM/YYYY)'])
            except Exception as e:
                print(e)
        if row['Typ'] != 'False':
            type_vocab, _ = SkosConcept.objects.get_or_create(
                pref_label="{}".format(row['Typ'])
            )
            type_vocab.scheme.add(res_type_scheme)
            item.res_type = type_vocab
        if row['Orte'] != 'False':
            for x in row['Orte'].split(';'):
                try:
                    pl = Place.objects.get(name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_place.add(pl)
        if row['Orte'] != 'False':
            for x in row['Orte'].split(';'):
                try:
                    pl = Place.objects.get(name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_place.add(pl)
        if row['Institutionen'] != 'False':
            for x in row['Institutionen'].split(';'):
                try:
                    pl = Institution.objects.get(written_name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_inst.add(pl)
        if row['Personen'] != 'False':
            for x in row['Personen'].split(';'):
                try:
                    pl = Person.objects.get(written_name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_person.add(pl)
        item.archiv = archiv
        item.save()

In [ ]:
# RepoLocation.objects.all().delete()